
<font color=orange>Extracting Subduction Zone Kinematics & Parameters (pyGPlates method)</font>

<font color=blue>Notebook 1</font>

This notebook can be used to extract subduction zone kinematics and parameters from a chosen plate model. These include:

overriding and subducting plate ID's
arc length
lat and lon
time
convergence rate and obliquity
migration rate and obliquity
age of the subducting seafloor
distance to nearest continent
It illustrates usage of the updated subduction zone kinematics pygplates method, using the Müller et al AREPS model:

implemented to work directly on the plate model gpml files (whereas before, you had to export the resolved topologies at 1Myr increments first)
this notebook also interpolates the age of the subducting plate age from age grids if provided
within the notebook data are stored in pandas dataframes, and there is an illustration of how to export them to both csv or hdf5 files
This was the first in a series of notebooks used to generate the data presened in the following manuscript:

'Subduction history reveals Cretaceous slab superflux as a possible cause for the mid-Cretaceous plume pulse and superswell events' by Madison East, R Dietmar Müller, Simon Williams, Sabin Zahirovic and Christian Heine

(Notebook written by Simon Williams with minor modifications made by Madison East, then Behnam Sadeghi)

<font color=green>Reference:</font>
East, M., Müller, R.D., Williams, S., Zahirovic, S., and Heine, C., 2020, Subduction history reveals Cretaceous slab superflux as a possible cause for the mid-Cretaceous plume pulse and superswell events, Gondwana Research, 79, 125-139.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import pygplates
import sys
sys.path.append('../../libs/')
import subduction_convergence as sc #(required code stored in a .py file)
import numpy as np
import os
os.environ['PROJ_LIB'] = r'C:\ProgramData\Anaconda3\pkgs\proj4-5.1.0-hfa6e2cd_1\Library\share'
import matplotlib.pyplot as plt
from matplotlib import gridspec

import scipy.interpolate as spi
from mpl_toolkits.basemap import Basemap
import inpaint
import pandas as pd

%matplotlib inline

In [2]:
import urllib
# The list array here specifies the age grid files, 
# assuming that the age grids are all together in a folder, with common filenames that
# differ only in the integer number somewhere in the filename that corresponds to the age in Ma
# The two parts of the list specify the parts of the filename before and after the number
# For example:
# ['/path_to_age_grids/age_grids_at_time_','_Ma.grd]
#agegrid_filename = ['../AgeGrids_most_recent/agegrid_final_nomask_','.grd']
agegrid_filename =  ["Muller_etal_2016_AREPS_v1.17_AgeGrid-",".nc"]
for time in range(0,231):
    fn = agegrid_filename[0] + str(time) + agegrid_filename[1]
    url = "https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/" + os.path.basename(fn)
    print(url)
    
    if not os.path.isfile(fn):
        filename, headers = urllib.request.urlretrieve (url, fn)
        print('Download ' + filename)

https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-0.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-0.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-1.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-1.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-2.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-2.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1

Download Muller_etal_2016_AREPS_v1.17_AgeGrid-29.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-30.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-30.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-31.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-31.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-32.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-32.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Mulle

Download Muller_etal_2016_AREPS_v1.17_AgeGrid-59.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-60.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-60.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-61.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-61.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-62.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-62.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Mulle

Download Muller_etal_2016_AREPS_v1.17_AgeGrid-89.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-90.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-90.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-91.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-91.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-92.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-92.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Mulle

Download Muller_etal_2016_AREPS_v1.17_AgeGrid-119.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-120.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-120.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-121.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-121.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-122.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-122.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrid

Download Muller_etal_2016_AREPS_v1.17_AgeGrid-149.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-150.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-150.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-151.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-151.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-152.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-152.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrid

Download Muller_etal_2016_AREPS_v1.17_AgeGrid-179.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-180.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-180.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-181.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-181.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-182.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-182.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrid

Download Muller_etal_2016_AREPS_v1.17_AgeGrid-209.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-210.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-210.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-211.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-211.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-212.nc
Download Muller_etal_2016_AREPS_v1.17_AgeGrid-212.nc
https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrid

In [3]:
from netCDF4 import Dataset
# This cell loads in the required files for the chosen plate model and defines some required functions #
#########################################################################################################

###################################################################
# Define Input files for Muller 2016 AREPS model (or chosen model)
###################################################################
# Specify the name of the GPlates rotation file
rotation_filename = 'Data/Global_EarthByte_230-0Ma_GK07_AREPS.rot'

# input topologies to be used with the subduction_convergence script
input_topology_filename = ['Data/Global_EarthByte_230-0Ma_GK07_AREPS_PlateBoundaries.gpmlz',\
             'Data/Global_EarthByte_230-0Ma_GK07_AREPS_Topology_BuildingBlocks.gpmlz']


# Static polygons to determine whether subduction segment is adjacent to continent or not
static_polygon_filename = 'Data/StaticPolygons/Global_EarthByte_GPlates_PresentDay_StaticPlatePolygons_2015_v1.shp'
static_polygon_features = pygplates.FeatureCollection(static_polygon_filename)
continental_polygon_features = []
for feature in static_polygon_features:
    if feature.get_feature_type() == pygplates.FeatureType.gpml_closed_continental_boundary:
        continental_polygon_features.append(feature)


######################################################
rotation_model = pygplates.RotationModel(rotation_filename)

# specify time range and resolution for plots
threshold_sampling_distance_radians = np.radians(0.5)

# Define the time snapshots at which to get the subduction zone properties
min_time = 0.
max_time = 230.
time_step = 1.

# Set the delta time for velocity calculations
velocity_delta_time = 1.

# Typically the achor plate id should be 0
anchor_plate_id = 0


#################################################
# Included functions
#################################################
def sample_grid_using_scipy(x,y,grdfile):
    
    data=Dataset(grdfile,'r')
    try:
        lon = np.copy(data.variables['x'][:])
        lat = np.copy(data.variables['y'][:])
    except:
        lon = np.copy(data.variables['lon'][:])
        lat = np.copy(data.variables['lat'][:])
    
    Zg = data.variables['z'][:]
    
    test = inpaint.fill_ndimage(Zg)
    
    lut=spi.RectBivariateSpline(lon,lat,test.T)
    result = []
    for xi,yi in zip(x,y):
        result.append(lut(xi, yi)[0][0])
            
    return result


def get_nearest_continental_polygon(subduction_data,continental_polygons):

    nearest_continent_plate_id = []
    distance_to_nearest_continent = []
    
    for index,row in subduction_data.iterrows():
        
        min_distance_to_all_features = np.radians(180)
        nearest_continent = None        
        
        seed_point = pygplates.PointOnSphere(row['lat'], row['lon'])
        
        for polygon in continental_polygons:
            if polygon is not None:
                min_distance_to_feature = pygplates.GeometryOnSphere.distance(
                    polygon.get_reconstructed_geometry(),
                    seed_point,
                    min_distance_to_all_features,
                    geometry1_is_solid=True)

                # If the current geometry is nearer than all previous geometries then
                # its associated feature is the nearest feature so far.
                if min_distance_to_feature is not None:
                    min_distance_to_all_features = min_distance_to_feature
                    nearest_continent = polygon.get_feature().get_reconstruction_plate_id()
                    
        nearest_continent_plate_id.append(nearest_continent)
        distance_to_nearest_continent.append(min_distance_to_all_features*pygplates.Earth.mean_radius_in_kms)

    return nearest_continent_plate_id,distance_to_nearest_continent

## Number crunching section

In [4]:
%%capture capt --no-stdout
#capture the annoying warning messages from pygplates


# Create a dataframe that contains all subduction data for full sequence of time steps

# Data frame template defining the column names
DataFrameTemplate = ('lon','lat','conv_rate','conv_obliq','migr_rate',
                     'migr_obliq','arc_length','arc_azimuth',
                     'subducting_plate','overriding_plate','time')

# Create an empty dataframe to concatenate results to
df_AllTimes = pd.DataFrame(columns=DataFrameTemplate)

# list of reconstruction times
reconstruction_times = np.arange(min_time,max_time+1,time_step)

# Iterate over time steps
for reconstruction_time in reconstruction_times:
        
    # Get the subduction kinematics database 
    subduction_data = []
    output_data = sc.subduction_convergence(
                rotation_filename,
                input_topology_filename,
                threshold_sampling_distance_radians,
                reconstruction_time,
                velocity_delta_time,
                anchor_plate_id)

    # Make a flat list of subduction stats to input into the proximity test
    for data in output_data:
        subduction_data.append(data+(reconstruction_time,))
    
    # convert list array to dataframe
    df = pd.DataFrame(subduction_data, columns = DataFrameTemplate)
    
    ######################
    # Age grid sampling
    grdfile = '%s%0.0f%s' % (agegrid_filename[0],reconstruction_time,agegrid_filename[1])
    print('working on file %s' % grdfile)
    sample_result = sample_grid_using_scipy(df['lon'],df['lat'],grdfile)
    
    # Append interpolate seafloor ages to dataframe
    df['SeafloorAge'] = pd.Series(sample_result)
    
    
    ######################
    # Find nearest continental polygon
    reconstructed_continental_polygons = []
    pygplates.reconstruct(continental_polygon_features,
                          rotation_model,
                          reconstructed_continental_polygons,
                          reconstruction_time,
                          anchor_plate_id=anchor_plate_id)
    pid,dist = get_nearest_continental_polygon(df,reconstructed_continental_polygons)
    df['NearestContinentPID'] = pd.Series(pid)
    df['DistanceToContinent'] = pd.Series(dist)

    # append dataframe 
    df_AllTimes = df_AllTimes.append(df)

working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-0.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-1.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-2.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-3.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-4.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-5.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-6.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-7.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-8.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-9.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-10.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-11.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-12.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-13.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-14.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-15.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-16.nc
working

working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-139.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-140.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-141.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-142.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-143.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-144.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-145.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-146.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-147.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-148.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-149.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-150.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-151.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-152.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-153.nc
working on file Muller_etal_2016_AREPS_v1.17_AgeGrid-154.nc
working on file Muller_etal_2016_AREPS_v

## Options for saving/loading results to/from file¶

In [5]:
# Save to csv file (prefered file format)
df_AllTimes.to_csv('SubductionTable_clean_%0.0f_%0.0fMa.csv' % (min_time,max_time)) 

# Save to an hdf5 file
#df_AllTimes.to_hdf('SubductionTable_%0.0f_%0.0fMa.h5' % (min_time,max_time),'SubductionTable') 

# Example of loading previously calculated results from a hdf5 file as created using the line above
#df_AllTimes = pd.read_hdf('SubductionTable_0_230Ma.h5')

#plt.scatter(df.lon,df.lat,c=df.DistanceToContinent,edgecolors='')
#plt.show()

In [6]:
# Display a preveiw of the data we just created
df_AllTimes

DistanceToContinent  NearestContinentPID  SeafloorAge  arc_azimuth  \
0                0.000000              16130.0    53.251326     7.048196   
1              104.296523                806.0   121.699738   293.688940   
2             1675.109671                844.0    99.730991   196.384032   
3              147.636960                806.0   121.422921   293.547286   
4              170.352834                823.0   121.061575   293.408606   
5             1694.367534                844.0   100.619943   196.319209   
6                0.000000              16130.0    53.454553     7.537337   
7              147.946897                823.0   120.782956   293.272837   
8             1714.223219                844.0   101.457858   196.253802   
9              127.142407                823.0   120.136847   286.827791   
10             120.471870                823.0   119.284535   286.727432   
11            1729.657345                844.0   102.305064   216.467133   
12             122.785370                823.0   118.164092   286.629375   
13               0.000000              16130.0    53.738991   351.178527   
14             138.929008                823.0   117.307421   286.533569   
15            1741.126634                844.0   103.160629   216.396194   
16             155.227797                823.0   116.420428   286.439965   
17             171.516207                823.0   115.222828   286.348514   
18            1753.765845                844.0   103.733990   216.323616   
19             187.793148                823.0   114.274200   286.259172   
20               0.000000              16130.0    53.974142   351.750823   
21             204.057529                823.0   113.373201   286.171894   
22             220.308259                823.0   112.639647   286.086636   
23            1766.629566                844.0   102.967967   217.662059   
24             236.544247                823.0   111.409564   286.003357   
25             252.764403                823.0   110.539519   285.922017   
26            1779.511976                844.0   102.216863   217.591718   
27             268.967638                823.0   109.639880   285.842576   
28               0.000000              16130.0    54.620963   352.325350   
29             285.152861                823.0   108.432499   285.764999   
...                   ...                  ...          ...          ...   
1421           489.615617                454.0    64.060379   307.216333   
1422           480.770411                454.0    62.498025   307.660905   
1423           401.231601                806.0    66.364267   228.450125   
1424           474.007224                454.0    60.880548   308.116678   
1425           472.973525                454.0    59.206418   308.583974   
1426           430.164422                806.0    66.679465   227.238178   
1427           478.207708                454.0    57.473263   309.063118   
1428           489.508749                454.0    55.660390   309.554449   
1429           463.681158                806.0    66.993022   225.965959   
1430           506.470696                454.0    53.755933   310.058311   
1431           528.548827                454.0    51.842719   310.575057   
1432           490.199464                813.0    67.305882   224.630315   
1433           555.133048                454.0    49.939511   311.105048   
1434           585.610006                454.0    48.470870   311.648651   
1435           619.405354                454.0    46.888604   312.206243   
1436           481.628168                813.0    67.616651   223.228055   
1437           656.006431                454.0    45.393628   312.778205   
1438           694.970077                454.0    43.649538   313.364927   
1439           464.134391                813.0    67.925289   221.755994   
1440           735.921124                454.0    42.140142   313.966802   
1441           778.546021                454.0    40.3

In [7]:
print('Done!')

Done!
